# 12— COMPLETE ROSSMANN, ETHICAL ISSUES

[Course Notebook on Github](https://github.com/fastai/fastai/blob/master/courses/ml1/lesson5-nlp.ipynb)

## Lecture Notes
My goal is not to copy what it is taught in the course and in the notebook. Its just various notes and trying things for my own.
Extrapolation in the previous notebook.

## Lecture Personal Notes
* Continue the Rossman Competition
    * Apply if only no vectorized option available.
    * PD: zip numpy arrays > pd.iterrows() - 300 times faster
    * Relu - cuts negative values
    * Create arbitrary event timers - before and after an event
    * Time Series - Windowing Functions - Average over a time window - Moving Average.
    * PD - rolling - arbitrary window functions, min periods to use.
    * RMSPE - Root Mean Squared Percent Error - real world we care about more about ratios
    * log of dependant variable
    * Old ML: Control Complexity via number of parameters. Modern ML: Control Complexity via regularization (dropout or weight decay).
    * Are you creating a good validation set.
* General Review
    * One-hot enc vector * coef = get where 1 is - look up that thing in an array - computation shortcut.
    * Aproximate the NN with a tree (Hinton).
    * Statistical Significance - Nowadays main driver is data volume, but won't be practical significance.
    * Small dataset - adjust for statistical significance - get statistical significance by bootstrapping - randomly resample the data set a number of times.
    * One-hot encoding with Trees - important for getting right feature importance, but not really doing anything for the model.
    * Influence actions than predictive accuracy.

## Lecture Code

In [32]:
# my own imports
from glob import glob
import pandas as pd
import numpy as np
import os

In [33]:
PATH = 'datasets/rossman'

In [34]:
train = pd.read_csv(PATH + '/train.csv')
test = pd.read_csv(PATH + '/test.csv')
store = pd.read_csv(PATH + '/store.csv')

/home/fury/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
train['Date'].head()

0    2015-07-31
1    2015-07-31
2    2015-07-31
3    2015-07-31
4    2015-07-31
Name: Date, dtype: object

In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
Store            1017209 non-null int64
DayOfWeek        1017209 non-null int64
Date             1017209 non-null object
Sales            1017209 non-null int64
Customers        1017209 non-null int64
Open             1017209 non-null int64
Promo            1017209 non-null int64
StateHoliday     1017209 non-null object
SchoolHoliday    1017209 non-null int64
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [37]:
train['Date'] = pd.to_datetime(train['Date'], format='%Y-%m-%d')

In [38]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(train.Store.values,train[fld].values, train.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    train[pre+fld] = res

In [41]:
fld = 'SchoolHoliday'
train = train.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
train = train.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [40]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,AfterSchoolHoliday
1016095,1,2,2013-01-01,0,0,0,0,a,1,0.0
1014980,1,3,2013-01-02,5530,668,1,0,0,1,0.0
1013865,1,4,2013-01-03,4327,578,1,0,0,1,0.0
1012750,1,5,2013-01-04,4486,619,1,0,0,1,0.0
1011635,1,6,2013-01-05,4997,635,1,0,0,1,0.0
